# Simulation of maximal models

Here we simulate data from a _zero-correlation-parameter_ model and fit four models to these data.

The simulation requires the `DataArrays`,`DataFrames` and `MixedModels` packages. For later extension to a parallel simulation we also restrict the BLAS to using a single thread.

In [1]:
using DataArrays,DataFrames,Distributions,MixedModels
blas_set_num_threads(1)

The `mkdata` function creates a data frame of the condition, `C`, the subject identifier, `S`, the item identifier, `I` and a place-holder for the response, `y`.  The calls to `pool` create a `PooledDataArray`, which is similar to a `factor` in [R](http://www.R-project.org).

In [2]:
"""
Create a DataFrame with crossed grouping factors, `S` and `I`
with `M` and `N` levels, respectively.  The condition, `C`,
should be a vector of length 2.  The effect of all the `rep` calls
is similar to `expand.grid` in R.
"""
function mkdata(C::Vector,N,M)
    l = length(C)
    DataFrame(y = zeros(l*N*M),
              C = rep(C,N*M),
              I = rep(rep(compact(pool(collect(1:N))),fill(l,N)),M),
              S = rep(compact(pool(collect(1:M))),fill(l*N,M)))
end

const dat = mkdata([-0.5,0.5],20,50);
dump(dat)

DataFrames.DataFrame  2000 observations of 4 variables
  y: DataArrays.DataArray{Float64,1}(2000) [0.0,0.0,0.0,0.0]
  C: DataArrays.DataArray{Float64,1}(2000) [-0.5,0.5,-0.5,0.5]
  I: DataArrays.PooledDataArray{Int64,UInt8,1}(2000) [1,1,2,2]
  S: DataArrays.PooledDataArray{Int64,UInt8,1}(2000) [1,1,1,1]


## Creating model objects without fitting them

In the `MixedModels` package the `lmm` function creates the model object from a formula/data specification, but does not cause it to be fit.  Applying `fit!` to such an object results in fitting the model.

First we create a vector of the five models.  In the simulation loop we will simulate the response, fit the models then extract the statistics of interest.

(Here, the `const` designation means that the type won't change, allowing for type inference on global objects.  The actual contents of the model can change.) 

In [3]:
const mm = [lmm(y ~ 1+C + (1+C|S) + (1+C|I), dat),
    lmm(y ~ 1+C + (1|S) + (0+C|S) + (1|I) + (0+C|I), dat),
    lmm(y ~ 1+C + (1|S) + (0+C|S) + (1|I), dat),
    lmm(y ~ 1+C + (1|S) + (1|I) + (0+C|I), dat),
    lmm(y ~ 1+C + (1|S) + (1|I), dat)];

## Simulate a single response vector and fit the models

The `simulate!` function simulates a response from model, optionally specifying the parameter values, and refits the model.

In [4]:
srand(1234321)
simulate!(mm[end];β=[2000.,0.],σ=300.,θ=[1/3,1/3])

Linear mixed model fit by maximum likelihood
 logLik: -14319.961802, deviance: 28639.923604, AIC: 28649.923604, BIC: 28677.928117

Variance components:
           Variance  Std.Dev.  
 S         9613.782  98.049894
 I        13430.846 115.891526
 Residual 90557.086 300.927044
 Number of obs: 2000; levels of grouping factors: 50, 20

  Fixed-effects parameters:
             Estimate Std.Error   z value
(Intercept)   1947.41   30.1512   64.5882
C            -1.64719   13.4579 -0.122396


This is an extremely fast process with very little allocation of memory.

In [5]:
gc(); srand(1234321); @time simulate!(mm[end];β=[2000.,0.],σ=300.,θ=[1/3,1/3]);

  0.004728 seconds (77.83 k allocations: 1.211 MB)


Next we fit the other models to this simulated response

In [6]:
for i in 1:length(mm)-1
    refit!(mm[i],model_response(mm[end]))
end

and extract the deviance.

In [7]:
devs = map(deviance,mm);
show(devs)

[28639.54709415252,28639.87243125265,28639.923604359417,28639.872431282096,28639.923604358577]

In [8]:
np = map(npar,mm);
show(np)

[9,7,6,6,5]

The values of AIC and BIC can be derived from the deviances and the numbers of parameters.

In [9]:
aic = devs .+ 2np;
show(aic)

[28657.54709415252,28653.87243125265,28651.923604359417,28651.872431282096,28649.923604358577]

In [10]:
bic = devs .+ log(nobs(mm[1])) * np;
show(bic)

[28707.9552162884,28693.078748469445,28685.52901911667,28685.47784603935,28677.928116656287]

For this completely balanced data set, the estimates of the fixed-effects parameters are (essentially) constant

In [11]:
map(fixef,mm)

5-element Array{Array{Float64,1},1}:
 [1947.4134048655926,-1.6471877485772273]
 [1947.413404865672,-1.6471877485736635] 
 [1947.4134048656886,-1.6471877485736643]
 [1947.413404865657,-1.647187748573664]  
 [1947.4134048656992,-1.6471877485736643]

but the standard errors change

In [12]:
map(stderr,mm)

5-element Array{Array{Float64,1},1}:
 [30.15405395982253,13.940561214940995]
 [30.152158547174494,13.93955786540863]
 [30.151404175517662,13.45786587741703]
 [30.15217319176865,13.93983317686538] 
 [30.15122661693448,13.457866553058672]

The p-value for the Wald test of the fixed-effect for slope is

In [13]:
map(m->ccdf(Chisq(1),abs2(fixef(m)[2] / stderr(m)[2])),mm)

5-element Array{Float64,1}:
 0.905943
 0.905936
 0.902585
 0.905938
 0.902585

Breaking down the calculation a bit, `ccdf` is the complementary cumulative distribution function (i.e. the probability of exceeding the value).  The p-value for a two-tailed z test is the same as the probability of a $\chi^2$ distribution on 1 degree of freedom exceeding $z^2$.

## Comparing LRT for the slope with the Wald test

The Wald test is based on a single model fit.  We would expect that a likelihood ratio test (LRT), based on fitting both the null model and the alternative model, would have better properties.  To perform these tests, we fit the null models and compare the difference in the deviances to a $\chi^2_1$ distribution.

In [14]:
const nm = [lmm(y ~ 1 + (1+C|S) + (1+C|I), dat),
    lmm(y ~ 1 + (1|S) + (0+C|S) + (1|I) + (0+C|I), dat),
    lmm(y ~ 1 + (1|S) + (0+C|S) + (1|I), dat),
    lmm(y ~ 1 + (1|S) + (1|I) + (0+C|I), dat),
    lmm(y ~ 1 + (1|S) + (1|I), dat)];

In [15]:
for i in eachindex(nm)
    refit!(nm[i],model_response(mm[end]))
end
ndevs = map(deviance,nm);
show(ndevs)

[28639.56103413647,28639.886389969117,28639.93858507726,28639.886389969393,28639.938585092023]

In [16]:
ccdf(Chisq(1),ndevs .- devs)

5-element Array{Float64,1}:
 0.906014
 0.905951
 0.902586
 0.905951
 0.902586

The p-values for the Wald test are nearly identical to those of the likelihood ratio test and are somewhat easier to obtain.